Data Preprocessing

In [1]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch as t

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [2]:
!pip install -U torchtext==0.8.0

In [3]:
!pip install transformers

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
np.random.seed(500)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df1 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_train.csv")
df2 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_test.csv")
df3 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_val.csv")
df4 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/schizoaffective.csv")
df5 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/AlAnon.csv")
df6 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/BorderlinePDisorder_10.csv")
df7 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/alcoholicsanonymous.csv")
df8 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/OCD.csv")

In [8]:
df4.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df5.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df6.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df7.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df8.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)

In [9]:
df4 = df4[df4["body"] != "[deleted]"]
df5 = df5[df5["body"] != "[deleted]"]
df6 = df6[df6["body"] != "[deleted]"]
df7 = df7[df7["body"] != "[deleted]"]
df8 = df8[df8["body"] != "[deleted]"]

In [10]:
df4["class_id"] = [6]*len(df4)
df5["class_id"] = [7]*len(df5)
df6["class_id"] = [8]*len(df6)
df7["class_id"] = [7]*len(df7)
df8["class_id"] = [9]*len(df8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [11]:
df4.dropna(inplace=True)
df5.dropna(inplace=True)
df6.dropna(inplace=True)
df7.dropna(inplace=True)
df8.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [12]:
df4.rename(columns={"body":"post","id":"ID"}, inplace=True)
df5.rename(columns={"body":"post","id":"ID"}, inplace=True)
df6.rename(columns={"body":"post","id":"ID"}, inplace=True)
df7.rename(columns={"body":"post","id":"ID"}, inplace=True)
df8.rename(columns={"body":"post","id":"ID"}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8]
corpus = pd.concat(frames, ignore_index=True)

In [14]:
corpus

,ID,title,post,class_name,class_id
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5
...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9


In [15]:
#df1.value_counts()

In [16]:
corpus["text"] = corpus.title + " " + corpus.post

In [17]:
corpus

,ID,title,post,class_name,class_id,text
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5,a question about the third conditional. i was ...
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0,the epitome of my life i've recently requested...
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5,what are your favourites offbeat destinations ...
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5,synesthesia survey (what colour is each month ...
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5,"science ama series: i’m phil baran, and i’m he..."
...,...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9,Can you have ocd without having depression??? ...
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9,"Hi! I'm an OCD therapist, and I'm new to reddi..."
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9,Vent: my mom who I still live with tested posi...
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9,Do “intrusive memories” count as intrusive tho...


In [18]:
corpus1 = corpus[["text","class_id"]]

In [19]:
corpus1

,text,class_id
0,a question about the third conditional. i was ...,5
1,the epitome of my life i've recently requested...,0
2,what are your favourites offbeat destinations ...,5
3,synesthesia survey (what colour is each month ...,5
4,"science ama series: i’m phil baran, and i’m he...",5
...,...,...
23572,Can you have ocd without having depression??? ...,9
23573,"Hi! I'm an OCD therapist, and I'm new to reddi...",9
23574,Vent: my mom who I still live with tested posi...,9
23575,Do “intrusive memories” count as intrusive tho...,9


In [20]:
corpus1["class_id"].value_counts()

7    4985
0    2961
3    2946
1    2918
2    2903
4    2497
5    2478
9     787
8     639
6     463
Name: class_id, dtype: int64

In [21]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [22]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(corpus['text'], corpus['class_id'], test_size=0.2, shuffle=True)

In [23]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 750 # max number of words in a question to use
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [24]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

## Pad the sentences 
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test= pad_sequences(x_test, maxlen=maxlen)

In [25]:
print("X_test: ",x_train, x_train.shape, x_train.shape[1])
print("X_test: ",x_test)

X_test:  [[   0    0    0 ...    5 1331  176]
 [   0    0    0 ...   19  289  709]
 [   0    0    0 ...   10  514 2426]
 ...
 [   0    0    0 ...    9  404  503]
 [   0    0    0 ...   52  110    2]
 [   0    0    0 ...  198   47  341]] (18861, 750) 750
X_test:  [[   0    0    0 ...   23 4420 1344]
 [   0    0    0 ...    5 4412 1245]
 [   0    0    0 ... 1301  121  385]
 ...
 [   0    0    0 ...  233  888  266]
 [   0    0    0 ... 2270    3  475]
 [   0    0    0 ... 3499    6  690]]


In [26]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2021-12-09 01:33:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-09 01:33:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-09 01:33:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [27]:
## FUNCTIONS TAKEN FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index):
    EMBEDDING_FILE = '/content/glove.6B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [28]:
if debug:
    embedding_matrix = np.random.randn(120000,300)
else:
    embedding_matrix = load_glove(tokenizer.word_index)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
np.shape(embedding_matrix)

(54822, 300)

BiLSTM

In [30]:
import torch

In [31]:
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 64
        drp = 0.3
        n_classes = 10
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)


    def forward(self, x):
        #rint(x.size())
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

In [32]:
n_epochs = 20
model = BiLSTM()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()

# Load train and test in CUDA Memory
x_train2 = torch.tensor(x_train, dtype=torch.long).cuda()
y_train2 = torch.tensor(y_train.values, dtype=torch.long).cuda()
x_cv = torch.tensor(x_test, dtype=torch.long).cuda()
y_cv = torch.tensor(y_test, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train2, y_train2)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),10))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==y_test)/len(y_test)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/20 	 loss=1092.9823 	 val_loss=971.9786  	 val_acc=0.2065  	 time=15.69s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 2/20 	 loss=953.2772 	 val_loss=745.0120  	 val_acc=0.4623  	 time=15.57s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

Epoch 18/20 	 loss=203.4776 	 val_loss=285.6355  	 val_acc=0.8204  	 time=15.96s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 19/20 	 loss=191.2727 	 val_loss=291.1041  	 val_acc=0.8170  	 time=15.96s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 20/20 	 loss=179.6026 	 val_loss=286.9780  	 val_acc=0.8179  	 time=15.98s


In [33]:
#torch.save(model,'bilstm_model.csv')